In [ ]:
# example from https://github.com/huggingface/transformers

In [1]:
import tensorflow as tf
import tensorflow_datasets
from transformers import *

In [ ]:
# Load dataset, tokenizer, model from pretrained model/vocabulary
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')
data = tensorflow_datasets.load('glue/mrpc')

In [ ]:
# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)

In [ ]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [6]:
# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
                    validation_data=valid_dataset, validation_steps=7)

# Load the TensorFlow model in PyTorch for inspection
model.save_pretrained('./save/')
pytorch_model = BertForSequenceClassification.from_pretrained('./save/', from_tf=True)

Epoch 1/2
115/115 [==============================] - 4752s 41s/step - loss: 0.5224 - accuracy: 0.7462 - val_loss: 0.4167 - val_accuracy: 0.8039
Epoch 2/2
115/115 [==============================] - 5059s 44s/step - loss: 0.2546 - accuracy: 0.9027 - val_loss: 0.6574 - val_accuracy: 0.7966


All TF 2.0 model weights were used when initializing BertForSequenceClassification.

Some weights of BertForSequenceClassification were not initialized from the TF 2.0 model and are newly initialized: ['bert.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Quickly test a few predictions - MRPC is a paraphrasing task, let's see if our model learned the task
sentence_0 = "This research was consistent with his findings."
sentence_1 = "His findings were compatible with this research."
sentence_2 = "His findings were not compatible with this research."
inputs_1 = tokenizer(sentence_0, sentence_1, add_special_tokens=True, return_tensors='pt')
inputs_2 = tokenizer(sentence_0, sentence_2, add_special_tokens=True, return_tensors='pt')

pred_1 = pytorch_model(inputs_1['input_ids'], token_type_ids=inputs_1['token_type_ids'])[0].argmax().item()
pred_2 = pytorch_model(inputs_2['input_ids'], token_type_ids=inputs_2['token_type_ids'])[0].argmax().item()

print("sentence_1 is", "a paraphrase" if pred_1 else "not a paraphrase", "of sentence_0")
print("sentence_2 is", "a paraphrase" if pred_2 else "not a paraphrase", "of sentence_0")

sentence_1 is a paraphrase of sentence_0
sentence_2 is a paraphrase of sentence_0


In [ ]:
# fine tuning would be the next stage - I havent done this
# Or I could have switched in one of these models instead
# BertForPreTraining, BertForNextSentencePrediction

In [ ]:
# Turning to pipeline

In [57]:
nlp1=pipeline('question-answering', model='distilbert-base-cased-distilled-squad')

In [58]:
nlp1({
     'question': 'What is the name of the repository ?',
     'context': 'Pipeline have been included in the huggingface/transformers repository'
 })

{'score': 0.5135946869850159,
 'start': 35,
 'end': 59,
 'answer': 'huggingface/transformers'}

In [60]:
nlp1({
    'question':'How are ideas narrowed down ?',
    'context': 'Our consultants work collaboratively with you to envision a future state, supported by the generation of new ideas. Those ideas are then narrowed down to a select few through workshops, which are then matured into separate Proof of Concept initiatives.'
})

{'score': 0.5169662833213806, 'start': 175, 'end': 185, 'answer': 'workshops,'}